In [1]:
import requests
import json
from loguru import logger

# Configure the base URL for your FastAPI server
BASE_URL = "http://127.0.0.1:8000"

def test_query_endpoint(query_text: str) -> dict:
    """
    Test the /query endpoint of the FastAPI server.
    
    Args:
        query_text: The text query to send to the manager agent.
        
    Returns:
        The response from the server.
    """
    url = f"{BASE_URL}/query"
    payload = {"text": query_text}
    
    try:
        logger.info(f"Sending request to {url}")
        logger.info(f"Payload: {payload}")
        
        response = requests.post(url, json=payload, timeout=30)
        response.raise_for_status()  # Raise an exception for bad status codes
        
        result = response.json()
        logger.info(f"Response received: {result}")
        return result
        
    except requests.exceptions.RequestException as e:
        logger.error(f"Request failed: {e}")
        return {"error": str(e)}
    except json.JSONDecodeError as e:
        logger.error(f"Failed to decode JSON response: {e}")
        return {"error": f"Invalid JSON response: {e}"}

def test_health_endpoint() -> dict:
    """
    Test the /health endpoint to check if the server is running.
    
    Returns:
        The health check response.
    """
    url = f"{BASE_URL}/health"
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        
        result = response.json()
        logger.info(f"Health check response: {result}")
        return result
        
    except requests.exceptions.RequestException as e:
        logger.error(f"Health check failed: {e}")
        return {"error": str(e)}

# Test the health endpoint first
logger.info("Testing health endpoint...")
health_response = test_health_endpoint()
print("Health Check Response:")
print(json.dumps(health_response, indent=2))


2025-06-09 15:59:15.686 | INFO     | __main__:<module>:61 - Testing health endpoint...
2025-06-09 15:59:15.707 | INFO     | __main__:test_health_endpoint:53 - Health check response: {'status': 'healthy', 'message': 'FastAPI server is running'}


Health Check Response:
{
  "status": "healthy",
  "message": "FastAPI server is running"
}


In [2]:
# Test the /query endpoint with example queries
print("\n" + "="*50)
print("Testing /query endpoint")
print("="*50)

# Example queries to test
test_queries = [
    "Why is the sky blue?",
]

for i, query in enumerate(test_queries, 1):
    print(f"\n--- Test Query {i} ---")
    print(f"Query: {query}")
    
    response = test_query_endpoint(query)
    print("Response:")
    print(json.dumps(response, indent=2))
    print("-" * 30)

2025-06-09 16:00:14.989 | INFO     | __main__:test_query_endpoint:22 - Sending request to http://127.0.0.1:8000/query
2025-06-09 16:00:14.993 | INFO     | __main__:test_query_endpoint:23 - Payload: {'text': 'Why is the sky blue?'}
2025-06-09 16:00:15.022 | ERROR    | __main__:test_query_endpoint:33 - Request failed: 500 Server Error: Internal Server Error for url: http://127.0.0.1:8000/query



Testing /query endpoint

--- Test Query 1 ---
Query: Why is the sky blue?
Response:
{
  "error": "500 Server Error: Internal Server Error for url: http://127.0.0.1:8000/query"
}
------------------------------


In [ ]:
# Interactive testing - uncomment and modify the query below to test with custom input
print("\n" + "="*50)
print("Interactive Testing")
print("="*50)

# Custom query for testing
custom_query = "Your custom query here"

print(f"Testing custom query: {custom_query}")
response = test_query_endpoint(custom_query)
print("Response:")
print(json.dumps(response, indent=2))

# Alternative: Test with input() for truly interactive testing
# Uncomment the lines below if you want to input queries interactively
"""
while True:
    user_input = input("\nEnter your query (or 'quit' to exit): ")
    if user_input.lower() == 'quit':
        break
    
    response = test_query_endpoint(user_input)
    print("Response:")
    print(json.dumps(response, indent=2))
"""


In [ ]:
# Debug: Check if agent servers are running
print("\n" + "="*50)
print("Debugging Agent Server Connection")
print("="*50)

def check_agent_server(port: int, name: str) -> bool:
    """Check if an agent server is running on the specified port."""
    try:
        response = requests.get(f"http://127.0.0.1:{port}", timeout=5)
        logger.info(f"{name} server (port {port}): Running ✓")
        return True
    except requests.exceptions.RequestException as e:
        logger.error(f"{name} server (port {port}): Not running ✗ - {e}")
        return False

# Check common agent server ports (adjust these based on your setup)
agent_ports = [
    (8002, "Manager Agent"),
    (8003, "Agent 1"), 
    (8004, "Agent 2"),
    (8005, "Agent 3")
]

print("Checking agent servers...")
all_running = True
for port, name in agent_ports:
    is_running = check_agent_server(port, name)
    if not is_running:
        all_running = False

if not all_running:
    print("\n🚨 Some agent servers are not running!")
    print("💡 Solution: Run './start_agents.sh' in your terminal to start the agent servers")
else:
    print("\n✅ All agent servers appear to be running!")


In [ ]:
# Enhanced error debugging for /query endpoint
print("\n" + "="*50)
print("Enhanced Query Testing with Error Details")
print("="*50)

def test_query_with_details(query_text: str) -> dict:
    """
    Test the /query endpoint with enhanced error reporting.
    
    Args:
        query_text: The text query to send to the manager agent.
        
    Returns:
        The response from the server with detailed error information.
    """
    url = f"{BASE_URL}/query"
    payload = {"text": query_text}
    
    try:
        logger.info(f"Sending request to {url}")
        logger.info(f"Payload: {payload}")
        
        response = requests.post(url, json=payload, timeout=30)
        
        # Log the status code and response content for debugging
        logger.info(f"Status Code: {response.status_code}")
        logger.info(f"Response Headers: {dict(response.headers)}")
        
        if response.status_code != 200:
            # Try to get the error details from the response
            try:
                error_details = response.json()
                logger.error(f"Server returned error: {error_details}")
                return {
                    "error": f"HTTP {response.status_code}",
                    "details": error_details,
                    "status_code": response.status_code
                }
            except json.JSONDecodeError:
                # If JSON parsing fails, return the raw text
                error_text = response.text
                logger.error(f"Server returned non-JSON error: {error_text}")
                return {
                    "error": f"HTTP {response.status_code}",
                    "details": error_text,
                    "status_code": response.status_code
                }
        
        # Success case
        result = response.json()
        logger.info(f"Response received: {result}")
        return result
        
    except requests.exceptions.RequestException as e:
        logger.error(f"Request failed: {e}")
        return {"error": f"Request failed: {str(e)}"}
    except json.JSONDecodeError as e:
        logger.error(f"Failed to decode JSON response: {e}")
        return {"error": f"Invalid JSON response: {e}"}

# Test with enhanced error reporting
test_query = "Why is the sky blue?"
print(f"Testing query with enhanced error reporting: {test_query}")
detailed_response = test_query_with_details(test_query)
print("Detailed Response:")
print(json.dumps(detailed_response, indent=2))
